# Sedaro Studies Example Notebook 
Demonstrates Sedaro Studies support via the sedaro python client using a Jupyter notebook.

# Introduction  
This notebook demonstrates how to use the new Studies support added to Sedaro. 
The core of this support is the new API python client object **SimStudy**.
**SimStudy** will generate and run a series of *SimJobs* in parallel up to account capacity limits. The remaining simjobs will be placed in a queue and will be execute when resources are available. 

Each *SimJob* of a **SimStudy** will set the random seed to a different value in order to generate different output results from the simulation.
The **Overrides** feature takes this a step further by providing a means to adjust any starting value of any model of the simulation using a set of pre-defined functions. In other words, each *SimJob* of a **SimStudy** will run a baseline Scenario branch with model parameters variations in order to observe their effects on performance results.

# Setup
Running this notebook requires the following:
- A Sedaro Account             --> https://www.sedaro.com
- A Sedaro API Token           --> https://www.sedaro.com/#/account
- The Wildfire Demo Branch ID  --> login --> select/click or create workspace --> select/click Project: [DEMO] WildFire --> select/click  Repositories: [DEMO] Wildfire *Scenarios* --> copy main branch ID via clipboard icon (TODO insert screen shot)
- The Wildfire Agent branch ID --> login --> select/click or create workspace --> select/click Project: [DEMO] WildFire --> select/click  Repositories: [DEMO] Wildfire *Vehicle* --> copy main branch ID via clipboard icon (TODO insert screen shot)
- Python 3.10+ installed       --> https://www.python.org
- Jupyter notebook or lab      --> https://jupyter.org

## Pip requirements
Create/activate a python venv if desired

In [ ]:
#!python -m venv /path/to/new/virtual/environment

Activate it via this table
| Platform | Shell | Command to activate virtual environment |
| :- | :- | :- |
| POSIX | bash/zsh |  source <venv>/bin/activate |
| | fish |  source <venv>/bin/activate.fish |
| | csh/tcsh |  source <venv>/bin/activate.csh |
| | PowerShell |  <venv>/bin/Activate.ps1 |
| Windows | cmd.exe | C:\> <venv>\Scripts\activate.bat |
| | PowerShell | PS C:\> <venv>\Scripts\Activate.ps1 |


Required python modules:
> replace *python* with *python3* if needed 


In [ ]:
#!python -m pip install -e sedaro pytest matplotlib pandas sweetviz
#!python -m pip install  IProgress flatten_json 'fuzzywuzzy[speedup]' python-Levenshtein

or

In [ ]:
#!python3.10 -m pip install  sedaro pytest matplotlib pandas sweetviz
#!python3.10 -m pip install  IProgress flatten_json 'fuzzywuzzy[speedup]' python-Levenshtein

Run the following cells to test if all required python modules are installed correctly

In [1]:
import sedaro
import yaml
import json
import matplotlib.pyplot as plt
import pandas as pd
import math
import numpy as np
import sweetviz as sv

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Important: Read Before Running

This notebook makes changes to agent and scenario branches indicated in the settings section. Ensure any changes to the target branches are saved prior to running this code. Sedaro recommends committing current changes and creating new branches in the target repositories to avoid loss of work.

This notebook also requires that you have previously generated an API key in the web UI. That key should be stored in a file called `secrets.json` in the same directory as this notebook with the following format:

```json
{
    "API_KEY": "<API_KEY>"
}
```

API keys grant full access to your repositories and should never be shared. If you think your API key has been compromised, you can revoke it in the user settings interface on the Sedaro website.


## Sedaro python client setup
Note: More information about the sedaro-python client can be found here: [https://github.com/sedaro/sedaro-python]


In the next cell, adjust the following variables as needed
- *Sedaro_api_host*
- *Sedaro_api_token*

In [2]:
Sedaro_api_host  = "http://localhost:80" # "api.sedaro.com"

# Set your API token value either directly or via loading a secrets file
# !! NOTE !!  Be careful not to check in your API Key into a source control repo  !! Note !!
secretPath = '/Users/sedaro/Documents/sedaro/sedaro-satellite/secrets.json'
with open(secretPath, 'r') as file:
    Sedaro_api_token = json.load(file)['API_KEY']
    
sedaroAPI = sedaro.SedaroApiClient(api_key=Sedaro_api_token, host=Sedaro_api_host)

## Load the wildfire scenario and wildfire agent branch data
Change the value of the *scenario_branch_id* and the *wildfire_agent_branch_id* in the next cell to the branch id noted above during the **Setup** section.
then run the following cell:

In [3]:
scenario_branch_id = "PLS7ksR9fHWl98HnYPZRK8"
wildfire_scenario_branch = sedaroAPI.scenario(scenario_branch_id)

wildfire_agent_branch_id = "PLS7kqQpYyy5qHXCVVByPQ"
wildfire_agent_branch = sedaroAPI.agent_template(wildfire_agent_branch_id)

### Find model parameter path 
Looking at the Wildfire scenario, it uses the wildfire template so lets search for the parameter paths we are interested in:

In [4]:
from sedaro import AgentModelParametersOverridePaths
wildfire_agent_paths = AgentModelParametersOverridePaths(wildfire_scenario_branch, wildfire_agent_branch, agent_name='Wildfire')

First search for **Reaction wheels (RW-XYZ)**

In [5]:
wildfire_agent_paths.findPathMatches("gain", 20)

['Wildfire/Imaging/id',
 'Wildfire/Imaging/name',
 'Wildfire/Imaging/type',
 'Wildfire/Operating/id',
 'Wildfire/GPS/dataSinks',
 'Wildfire/GPS/hotMargin',
 'Wildfire/Magnetometer/dataSinks',
 'Wildfire/Magnetometer/hotMargin',
 'Wildfire/Attitude Control/gainC',
 'Wildfire/Attitude Control/gainG',
 'Wildfire/Attitude Control/gainK',
 'Wildfire/data/position',
 'Wildfire/Battery/minSoc',
 'Wildfire/GPS/coldMargin',
 'Wildfire/Magnetometer/coldMargin',
 'Wildfire/Gyro/dataSinks',
 'Wildfire/Gyro/hotMargin',
 'Wildfire/MT-X/dataSinks',
 'Wildfire/MT-X/hotMargin',
 'Wildfire/MT-Y/dataSinks']

#### Create and run Study
Uncomment the following to start the study job

In [6]:
create_study_resource_url = f'/simulations/branches/{scenario_branch_id}/control/study/'
# new_studyjob = sedaroAPI.request.post(  create_study_resource_url,
#                                         body={
#                                             "iterations": 6,
#                                             #"override_id": monte_carlo_overrides_id
#                                             })
new_studyjob = { "id": "PLVFyTr9zTFLGycLJLKPf8"} 
new_studyjob

{'id': 'PLVFyTr9zTFLGycLJLKPf8'}

Generates a list of all the study jobs in your workspace and its status

In [7]:
[ (study['id'], study['status']) for study in sedaroAPI.request.get(  create_study_resource_url) ]

[('PLVFyTr9zTFLGycLJLKPf8', 'SUCCEEDED'), ('PLVCgkW4H8PYWzhH3XYZGj', 'ERROR')]

### Check on Study Status
Either set the variable to the correct ID or uncomment to set it from the new_studyjob object

In [8]:
#study_id = 'PLVCgkW4H8PYWzhH3XYZGj' 
study_id =  new_studyjob['id'] #
study_control_resource = f'/simulations/branches/{scenario_branch_id}/control/study/{study_id}'
study_status = sedaroAPI.request.get(study_control_resource)
study_status

{'branch': 'PLS7ksR9fHWl98HnYPZRK8',
 'dateCreated': '2024-02-12T18:50:56.823Z',
 'dateModified': '2024-02-12T19:14:28.191Z',
 'id': 'PLVFyTr9zTFLGycLJLKPf8',
 'jobType': 'STUDY',
 'jobs': ['PLVFyYKWXhCHrmnYYJCtjn',
  'PLVCgvvMD7nmjJbS69GvFh',
  'PLVCh342C5DN7WXRLBtMDv',
  'PLVCh2ZHpS8GJ5tVRW2bLl',
  'PLVCgsgPk66vnKrRg6GDsd',
  'PLSVkDtZMjjv8vCJzlHMPG'],
 'message': 'Started study successfully.',
 'predecessor': None,
 'scenarioHash': 'b4d77b648e5bccf714a124a864cb0e890eecbde107e711719318d069d439e375',
 'status': 'SUCCEEDED',
 'successors': [],
 'versions': {'sedaro-app': 'dev'},
 'workspace': 'PLS7kZdD3PsNjjv4R4pySm'}

List the status of all the sim jobs of the study job

In [9]:
study_job_ids = study_status['jobs']
[ ( f"SimJob ID: {job['id']}", f"Status: {job['status']}", f"Progress:", job['progress']) for job_id in study_job_ids for job in [sedaroAPI.request.get(f'/simulations/branches/{scenario_branch_id}/control/{job_id}')] ]

[('SimJob ID: PLVFyYKWXhCHrmnYYJCtjn',
  'Status: SUCCEEDED',
  'Progress:',
  {'currentTime': None, 'percentComplete': 100}),
 ('SimJob ID: PLVCgvvMD7nmjJbS69GvFh',
  'Status: SUCCEEDED',
  'Progress:',
  {'currentTime': None, 'percentComplete': 100}),
 ('SimJob ID: PLVCh342C5DN7WXRLBtMDv',
  'Status: SUCCEEDED',
  'Progress:',
  {'currentTime': None, 'percentComplete': 100}),
 ('SimJob ID: PLVCh2ZHpS8GJ5tVRW2bLl',
  'Status: SUCCEEDED',
  'Progress:',
  {'currentTime': None, 'percentComplete': 100}),
 ('SimJob ID: PLVCgsgPk66vnKrRg6GDsd',
  'Status: SUCCEEDED',
  'Progress:',
  {'currentTime': None, 'percentComplete': 100}),
 ('SimJob ID: PLSVkDtZMjjv8vCJzlHMPG',
  'Status: SUCCEEDED',
  'Progress:',
  {'currentTime': None, 'percentComplete': 100})]

### Load the results of the Study SimJobs
Load the the Study Results then load the result data from all of its  simulation set. We'll reduce each sim result data set to 2000 points each for this example.

In [10]:
gb_study_job = wildfire_scenario_branch.study.results(study_status['id'])

In [11]:
gb_study_job.summarize() 

---------------------------------------------------------------------------
                        Sedaro Study Result Summary                        
                       Job ID PLVFyTr9zTFLGycLJLKPf8                       
---------------------------------------------------------------------------
✅ Study succeeded

📋 Study contains 6 simulations

 ['PLVFyYKWXhCHrmnYYJCtjn', 'PLVCgvvMD7nmjJbS69GvFh', 'PLVCh342C5DN7WXRLBtMDv', 'PLVCh2ZHpS8GJ5tVRW2bLl', 'PLVCgsgPk66vnKrRg6GDsd', 'PLSVkDtZMjjv8vCJzlHMPG']

❗ In-memory simulation result caching is ON
---------------------------------------------------------------------------
❓ Agent data not yet loaded. Load with .summarize_agents()
---------------------------------------------------------------------------
❓ First set desired results downsampling with:
       .set_result_limit(<# of points>) 
       .set_result_binWidth( <fraction of overall points> )   
---------------------------------------------------------------------------
❓ Qu

In [12]:
gb_study_job.summarize_agents()

💾 Downloading simulation result id PLVFyYKWXhCHrmnYYJCtjn...done!
💾 Downloading simulation result id PLVCgvvMD7nmjJbS69GvFh...done!
💾 Downloading simulation result id PLVCh342C5DN7WXRLBtMDv...done!
💾 Downloading simulation result id PLVCh2ZHpS8GJ5tVRW2bLl...done!
💾 Downloading simulation result id PLVCgsgPk66vnKrRg6GDsd...done!
💾 Downloading simulation result id PLSVkDtZMjjv8vCJzlHMPG...done!
PLVFyYKWXhCHrmnYYJCtjn:✅ Simulation succeeded after 1081.6s
PLVCgvvMD7nmjJbS69GvFh:✅ Simulation succeeded after 6934.9s
PLVCh342C5DN7WXRLBtMDv:✅ Simulation succeeded after 4032.3s
PLVCh2ZHpS8GJ5tVRW2bLl:✅ Simulation succeeded after 2784.6s
PLVCgsgPk66vnKrRg6GDsd:✅ Simulation succeeded after 1430.3s
PLSVkDtZMjjv8vCJzlHMPG:✅ Simulation succeeded after 1029.3s

🛰️ Templated Agents 
    • Wildfire
    • LaserComm-1
    • LaserComm-2
    • LaserComm-3
    • LaserComm-4

📡 Peripheral Agents 
    • Ground Station: Cape Canaveral
    • Fire: Brook Mole
    • Fire: Chichen Itza
    • Fire: Jasper National 

Work out propose Score function baseline values

In [13]:
study_Wildfire_agents = gb_study_job.agents('Wildfire')

In [15]:
study_Wildfire_agents.blocksname("RW-X").summarize()

---------------------------------------------------------------------------
                   Study Simulation Block Result Summary                   
                                   'RW-X'                                  
                     Study ID: 'PLVFyTr9zTFLGycLJLKPf8'                    
---------------------------------------------------------------------------
---------------------------------------------------------------------------
                           Study Simulation ID's                           

 ['PLVFyYKWXhCHrmnYYJCtjn', 'PLVCgvvMD7nmjJbS69GvFh', 'PLVCh342C5DN7WXRLBtMDv', 'PLVCh2ZHpS8GJ5tVRW2bLl', 'PLVCgsgPk66vnKrRg6GDsd', 'PLSVkDtZMjjv8vCJzlHMPG']
---------------------------------------------------------------------------
🧩 Simulated Modules
    • Command & Data Handling
    • Power
    • Guidance, Navigation, & Control
    • Thermal

📋 Variables Available
    • activeDataMode
    • dissipations
    • powerConsumed
    • activeFailureMode
    • comman

In [ ]:
study_Wildfire_agents.blocksname("RW-X").ratedMomentum.study_stats()

In [ ]:
study_Wildfire_agents.blocksname("RW-Y").ratedMomentum.study_stats()

In [ ]:
study_Wildfire_agents.blocksname("RW-Z").ratedMomentum.study_stats()

In [17]:
study_Wildfire_agents.blocksname("RW-X").momentum.study_stats()

,,count,mean,std,min,25%,50%,75%,max
PLVFyYKWXhCHrmnYYJCtjn,momentum,10079.0,-1.008978,3.232228,-8.0,-2.362933,-0.024883,0.276104,8.0
PLVCgvvMD7nmjJbS69GvFh,momentum,10070.0,-1.006253,3.237032,-8.0,-2.373001,-0.021990,0.284699,8.0
PLVCh342C5DN7WXRLBtMDv,momentum,10097.0,-0.999460,3.229194,-8.0,-2.356070,-0.030532,0.290668,8.0
PLVCh2ZHpS8GJ5tVRW2bLl,momentum,10095.0,-1.010724,3.231202,-8.0,-2.372393,-0.028740,0.276956,8.0
PLVCgsgPk66vnKrRg6GDsd,momentum,10094.0,-1.009860,3.231369,-8.0,-2.364316,-0.036300,0.272428,8.0
PLSVkDtZMjjv8vCJzlHMPG,momentum,10061.0,-0.931710,3.245517,-8.0,-2.115147,-0.016318,0.326247,8.0


In [18]:
study_Wildfire_agents.blocksname("RW-Y").momentum.study_stats()

,,count,mean,std,min,25%,50%,75%,max
PLVFyYKWXhCHrmnYYJCtjn,momentum,10079.0,-0.426672,4.123396,-8.0,-1.408662,-0.055275,0.566691,8.0
PLVCgvvMD7nmjJbS69GvFh,momentum,10070.0,-0.425711,4.132959,-8.0,-1.415514,-0.044944,0.569987,8.0
PLVCh342C5DN7WXRLBtMDv,momentum,10097.0,-0.416875,4.126847,-8.0,-1.405268,-0.065450,0.587590,8.0
PLVCh2ZHpS8GJ5tVRW2bLl,momentum,10095.0,-0.411778,4.128747,-8.0,-1.392407,-0.060196,0.570416,8.0
PLVCgsgPk66vnKrRg6GDsd,momentum,10094.0,-0.411273,4.129552,-8.0,-1.408675,-0.061643,0.576057,8.0
PLSVkDtZMjjv8vCJzlHMPG,momentum,10061.0,-0.182454,4.175412,-8.0,-1.012042,-0.039236,0.713625,8.0


In [19]:
study_Wildfire_agents.blocksname("RW-Z").momentum.study_stats()

,,count,mean,std,min,25%,50%,75%,max
PLVFyYKWXhCHrmnYYJCtjn,momentum,10079.0,-0.803394,3.998075,-8.0,-2.023735,-0.212640,0.206883,8.0
PLVCgvvMD7nmjJbS69GvFh,momentum,10070.0,-0.818657,3.992794,-8.0,-2.024394,-0.245619,0.174530,8.0
PLVCh342C5DN7WXRLBtMDv,momentum,10097.0,-0.801814,4.014107,-8.0,-2.051822,-0.243683,0.209647,8.0
PLVCh2ZHpS8GJ5tVRW2bLl,momentum,10095.0,-0.798247,4.009146,-8.0,-2.045464,-0.217130,0.177729,8.0
PLVCgsgPk66vnKrRg6GDsd,momentum,10094.0,-0.804115,3.998401,-8.0,-2.071608,-0.212974,0.196794,8.0
PLSVkDtZMjjv8vCJzlHMPG,momentum,10061.0,-1.062015,3.885358,-8.0,-2.699386,-0.237258,0.159640,8.0


In [23]:
study_Wildfire_agents.summarize()

---------------------------------------------------------------------------
                         Study Agent Result Summary                        
                                 'Wildfire'                                
                     Study ID: 'PLVFyTr9zTFLGycLJLKPf8'                    
---------------------------------------------------------------------------
                           Study Simulation ID's                           

 ['PLVFyYKWXhCHrmnYYJCtjn', 'PLVCgvvMD7nmjJbS69GvFh', 'PLVCh342C5DN7WXRLBtMDv', 'PLVCh2ZHpS8GJ5tVRW2bLl', 'PLVCgsgPk66vnKrRg6GDsd', 'PLSVkDtZMjjv8vCJzlHMPG']
---------------------------------------------------------------------------
🧩 Study Simulated Modules
    • Command & Data Handling
    • Power
    • Guidance, Navigation, & Control
    • Thermal

📦 Available Blocks
    ----------------------------------------------------------
    |                  id                         name       |
    ---------------------------------------

In [37]:
study_Wildfire_agents.blocks('root').pointingErrorAngle['deg'].study_stats()

,,count,mean,std,min,25%,50%,75%,max
PLVFyYKWXhCHrmnYYJCtjn,pointingErrorAngle.deg,10077.0,34.786806,43.419709,0.108700,1.628421,8.679727,63.330971,179.792142
PLVCgvvMD7nmjJbS69GvFh,pointingErrorAngle.deg,10068.0,34.703106,43.391442,0.170956,1.628830,8.565667,63.108677,179.503603
PLVCh342C5DN7WXRLBtMDv,pointingErrorAngle.deg,10095.0,34.859085,43.430362,0.120628,1.624585,8.875162,63.452844,179.733501
PLVCh2ZHpS8GJ5tVRW2bLl,pointingErrorAngle.deg,10093.0,34.879303,43.487587,0.081162,1.619676,8.830720,63.438808,179.368910
PLVCgsgPk66vnKrRg6GDsd,pointingErrorAngle.deg,10092.0,34.847433,43.465401,0.188075,1.623997,8.743328,63.439456,179.216083
PLSVkDtZMjjv8vCJzlHMPG,pointingErrorAngle.deg,10059.0,34.651030,43.319196,0.161697,1.619581,8.473820,63.149583,179.798531


Study 'Score'

In [39]:
weight_momentum_ratio = 1.0
weight_pointing_error = 1.0
max_rated_momentum    = 8.0
# rw_x_max_rated_momentum = study_Wildfire_agents.blocksname("RW-X").ratedMomentum.study_stats()['max']
# rw_y_max_rated_momentum = study_Wildfire_agents.blocksname("RW-Y").ratedMomentum.study_stats()['max']
# rw_z_max_rated_momentum = study_Wildfire_agents.blocksname("RW-Z").ratedMomentum.study_stats()['max']

rw_x_avg_momentum = study_Wildfire_agents.blocksname("RW-X").momentum.study_stats()['mean'] 
rw_y_avg_momentum = study_Wildfire_agents.blocksname("RW-Y").momentum.study_stats()['mean']
rw_z_avg_momentum = study_Wildfire_agents.blocksname("RW-Z").momentum.study_stats()['mean']

score_momentum_rw_x_ratio_squared = math.pow(rw_x_avg_momentum/max_rated_momentum, 2.0) 
score_momentum_rw_y_ratio_squared = math.pow(rw_y_avg_momentum/max_rated_momentum, 2.0)
score_momentum_rw_z_ratio_squared = math.pow(rw_z_avg_momentum/max_rated_momentum, 2.0)

score_momentum_component = weight_momentum_ratio * math.sqrt(score_momentum_rw_x_ratio_squared + score_momentum_rw_y_ratio_squared + score_momentum_rw_z_ratio_squared)
score_pointing_error_component = weight_pointing_error * study_Wildfire_agents.blocksname("data").pointingErrorAngle.study_stats()['avg']

final_score = score_momentum_component + score_pointing_error_component
final_score

TypeError: cannot convert the series to <class 'float'>

In [42]:
study_status['jobs']

['PLVFyYKWXhCHrmnYYJCtjn',
 'PLVCgvvMD7nmjJbS69GvFh',
 'PLVCh342C5DN7WXRLBtMDv',
 'PLVCh2ZHpS8GJ5tVRW2bLl',
 'PLVCgsgPk66vnKrRg6GDsd',
 'PLSVkDtZMjjv8vCJzlHMPG']

In [73]:
simjob_id = study_status['jobs'][0]
rw_x_momentum_df = study_Wildfire_agents.blocksname("RW-X").momentum.make_study_dataframe()
rw_x_momentum_df.describe().T['mean']['PLVFyYKWXhCHrmnYYJCtjn'].values[0]

-1.0089778047396432

In [90]:
def simjob_scores(study_jobs, study_agents):
    weight_momentum_ratio = 4.0
    weight_pointing_error = 1.0
    max_rated_momentum    = 8.0
    for simjob_id in study_jobs:
        rw_x_momentum_df = study_agents.blocksname("RW-X").momentum.make_study_dataframe()
        rw_x_avg_momentum = rw_x_momentum_df.describe().T['mean'][simjob_id].values[0]
        rw_y_momentum_df = study_agents.blocksname("RW-Y").momentum.make_study_dataframe()
        rw_y_avg_momentum = rw_y_momentum_df.describe().T['mean'][simjob_id].values[0]
        rw_z_momentum_df = study_agents.blocksname("RW-Z").momentum.make_study_dataframe()
        rw_z_avg_momentum = rw_z_momentum_df.describe().T['mean'][simjob_id].values[0]

        score_momentum_rw_x_ratio_squared = (rw_x_avg_momentum/max_rated_momentum)**2 
        score_momentum_rw_y_ratio_squared = (rw_y_avg_momentum/max_rated_momentum)**2
        score_momentum_rw_z_ratio_squared = (rw_z_avg_momentum/max_rated_momentum)**2

        score_momentum_component = weight_momentum_ratio * math.sqrt(score_momentum_rw_x_ratio_squared + score_momentum_rw_y_ratio_squared + score_momentum_rw_z_ratio_squared)

        pointing_error_df = study_agents.blocks("root").pointingErrorAngle['rad'].make_study_dataframe()
        score_pointing_error_component = weight_pointing_error * pointing_error_df.describe().T['mean'][simjob_id].values[0]

        final_score = score_momentum_component + score_pointing_error_component
        print(f"SimJob ID: {simjob_id}  Final Score: {final_score} Momentum Ratio: {score_momentum_component}  Pointing Error: {score_pointing_error_component}")


In [91]:
simjob_scores(study_status['jobs'], study_Wildfire_agents)

SimJob ID: PLVFyYKWXhCHrmnYYJCtjn  Final Score: 1.2863952910887124 Momentum Ratio: 0.6792509984993955  Pointing Error: 0.6071442925893168
SimJob ID: PLVCgvvMD7nmjJbS69GvFh  Final Score: 1.288320639344283 Momentum Ratio: 0.6826371760057066  Pointing Error: 0.6056834633385763
SimJob ID: PLVCh342C5DN7WXRLBtMDv  Final Score: 1.2821286797134026 Momentum Ratio: 0.6737228674578699  Pointing Error: 0.6084058122555326
SimJob ID: PLVCh2ZHpS8GJ5tVRW2bLl  Final Score: 1.2848357997408155 Momentum Ratio: 0.676077113510002  Pointing Error: 0.6087586862308136
SimJob ID: PLVCgsgPk66vnKrRg6GDsd  Final Score: 1.2856173996270122 Momentum Ratio: 0.6774149567532498  Pointing Error: 0.6082024428737625
SimJob ID: PLSVkDtZMjjv8vCJzlHMPG  Final Score: 1.3170328803630593 Momentum Ratio: 0.7122583126226634  Pointing Error: 0.604774567740396
